<img src="Imagenes/Mac_wallpaper_3.png" width="50%">

In [ ]:
!pip install "qiskit[visualization]" --user

In [ ]:
!pip install qiskit-aer --user

# Identificar gráficas bipartitas

Ahora, vamos a ver un ejemplo del uso del algoritmo de Grover como herramienta para resolver un problema concreto. Y para ello, vamos a utilizar una versión simplificada de un problema bastante conocido: el problema del corte máximo.

### El problema de corte máximo

Supongamos que tenemos la siguiente figura

<img src="Imagenes/cuadrado.png" width="20%">

En ella tenemos cuatro nodos o "esquinas", numeradas convenientemente de 0 a 3; además, tenemos cuatro líneas conectoras o "lados" en nuestra figura, que esencialmente es un cuadrado. Supongamos ahora que queremos marcar estos cuatro nodos de manera que queden separados en dos grupos: los que están pintados de rojo y los que están pintados de azul, por ejemplo. Esto nos permite hacer un "corte" en nuestra figura, el cual puede lucir de la siguiente manera

<img src="Imagenes/cuadrado_1.png" width="20%">

En este caso, la línea verde marca nuestro corte, el cual divide a nuestra figura en dos subconjuntos:

$\hspace{9 cm} A = \{0,1\} \hspace{3 cm} B = \{2,3\} $

Ahora, nos vamos a centrar en las líneas conectoras de nuestra figura y vamos a contar solamente las líneas que sean atravesadas por el delimitador de nuestro corte, es decir, por la línea verde. Estas son las que nos van a permitir identificar el "tamaño" de nuestro corte, el cual es de $2$ en este caso. 

Es decir, el tamaño de nuestro corte es el número de líneas conectoras que conectan dos nodos que pertenecen a subconjuntos diferentes de nuestra partición

Y el objetivo del problema es encontrar el corte de máximo tamaño, el cual en este caso es fácil ver que se trata de $4$.

Sin embargo, el circuito necesario para resolver un problema de corte máximo es un poco más complicado que los circuitos que hemos visto anteriormente, y requiere de varias subrutinas extras como sumadores y comparadores. Es por ello que nos vamos a dedicar a resolver una versión más sencilla de este problema: verificar si una gráfica es bipartita o no. 

### Gráficas bipartitas

Una gráfica bipartita es aquella que admite un corte en el cual **ninguna** línea conectora conecta dos nodos que pertenecen a un mismo subconjunto de la partición. 

Es decir, si tomamos la figura de la sección anterior como ejemplo veremos que esta es una gráfica bipartita: si coloreamos los nodos 0 y 3 de azul, y los nodos 1 y 2 de rojo, veremos que todas nuestras líneas conectoras van de un elemento rojo a uno azul.

### Codificación del problema

¡Muy bien! Ya sabemos que problema queremos resolver, ahora solo falta codificarlo en un circuito. Para ello, vamos a realizar los siguientes pasos:

- Vamos a asignar cada uno de los nodos a un qubit cada uno. Si el estado de nuestro qubit es 0, significa que el nodo correspondiente está pintado de azul; y si el estado de nuestro qubit es 1, entonces el nodo correspondiente está pintado de rojo. <br><br>
- Vamos a asignar cada una de las líneas conectoras a un qubit cada una también. De esta manera si el estado de un qubit que representa una línea conectora es 0, entonces esta línea conecta dos nodos que tienen el mismo color; y si el estado de un qubit que representa una línea conectora es 1, entonces esta línea conecta dos nodos que tienen un color diferente.<br><br>
- Vamos a utilizar un último qubit, el cual nos va a ayudar a verificar si nuestra partición cumple la condición que buscamos. Y esto se reduce a verificar que todos los qubits que representan líneas conectoras se encuentren en el estado 1.

De esta manera, si quisieramos resolver el caso de la gráfica que hemos usado como ejemplo utilizando el algoritmo de Grover, necesitaríamos: 4 qubits para los nodos, 4 qubits para las líneas conectoras y 1 qubit para verificar que tenemos la respuesta correcta. Es decir, necesitamos 9 qubits en total.

### Creación del oráculo

Como seguro recordarás de la construcción del algoritmo de Grover, el oráculo es el que se encarga de marcar la respuesta que estamos buscando, y por lo tanto, varía de problema en problema. En nuestro caso, necesitamos un oráculo que marque el (o los) estado(s) que representen una partición que cumpla los requisitos para considerarse una gráfica bipartita. Para ello, vamos a crear un oráculo que realice tres pasos:

- Revisar si una línea conectora conecta dos nodos que pertenecen a un mismo subconjunto, o no. Esto se puede hacer utilizando dos compuertas CNOT tomando los qubits de los nodos como controles y el qubit de la línea conectora como objetivo. De esta manera si ambos qubits de los nodos se encuentran en el estado 1, aplicaremos dos compuertas NOT al qubit de la línea conectora (esto lo deja en el estado 0); y si ambos qubits de los nodos se encuentran en el estado 0, no haremos nada.<br><br>
- Revisar que **todos** los qubits de las líneas conectoras estén en el estado 1. Esto se puede hacer con una compuerta Toffoli multicontrolada, tomando a los qubits de las líneas conectoras como controles y el qubit de verificación como el objetivo.<br><br>
- Marcar solo los estados cuyo qubit de verificación se encuentre en el estado 1. Esta parte es bastante sencilla; solo necesitamos aplicar una compuerta $Z$ sobre el qubit de verificación.

Ahora, creemos un oráculo para nuestro ejemplo. Recuerda siempre tener en cuenta qué qubit representa qué línea conectora.

In [ ]:
from qiskit import QuantumCircuit, execute, Aer, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram

def oracle(qc,nod,lin,ver):
#Primero, revisamos las líneas conectoras
    qc.cx(nod[0],lin[0]) #Empezamos con la línea conectora de los nodos 0-1
    qc.cx(nod[1],lin[0])
    
    qc.cx(nod[0],lin[1]) #Línea conectora de los nodos 0-2
    qc.cx(nod[2],lin[1])
    
    qc.cx(nod[1],lin[2]) #Línea conectora de los nodos 1-3
    qc.cx(nod[3],lin[2])
    
    qc.cx(nod[2],lin[3]) #Línea conectora de los nodos 2-3
    qc.cx(nod[3],lin[3])
    
#Ahora revisamos que todas las líneas conectoras se encuentren en el estado 1

    qc.mcx([lin[0],lin[1],lin[2],lin[3]],ver[0])

Antes de marcar el estado que nos interesa, verifiquemos que nuestro oráculo realiza lo que le pedimos.

In [ ]:
nod = QuantumRegister(4, name="nod")
lin = QuantumRegister(4, name="lin")
ver = QuantumRegister(1, name="ver")
c = ClassicalRegister(1)

qc = QuantumCircuit(nod,lin,ver,c)

#Creamos el estado que querramos verificar
#qc.x(nod[0])
#qc.x(nod[1])
#qc.x(nod[2])
#qc.x(nod[3])

#Aplicamos el oráculo

oracle(qc,nod,lin,ver)

#Medimos el registro de verificación

qc.measure(ver,c)

#Dibujamos nuestro circuito para ver cómo luce

qc.draw(output="mpl")

In [ ]:
#Ejecutamos nuestro circuito
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=10)
counts = job.result().get_counts(qc)
print(counts)

¡Perfecto! Ahora, antes de proseguir con el último paso de nuestro oráculo, hay un detalle que debemos tener en cuenta: vamos a aplicar el oráculo múltiples veces en el algoritmo de Grover, por lo que es necesario que nuestro qubit de verificación y nuestros qubits de las líneas conectoras regresen al estado 0 antes de pasar a la siguiente iteración. Para ello, vamos a crear una nueva función que aplicará el inverso de nuestro oráculo.

In [ ]:
def inverse_oracle(qc,nod,lin,ver):
#Primero, regresamos el qubit de verificación al estado 0
    qc.mcx([lin[0],lin[1],lin[2],lin[3]],ver[0])
    
#Y ahora, regresamos a los qubits de las líneas conectoras a 0 también
    qc.cx(nod[0],lin[0]) #Empezamos con la línea conectora de los nodos 0-1
    qc.cx(nod[1],lin[0])
    
    qc.cx(nod[0],lin[1]) #Línea conectora de los nodos 0-2
    qc.cx(nod[2],lin[1])
    
    qc.cx(nod[1],lin[2]) #Línea conectora de los nodos 1-3
    qc.cx(nod[3],lin[2])
    
    qc.cx(nod[2],lin[3]) #Línea conectora de los nodos 2-3
    qc.cx(nod[3],lin[3])

Y finalmente, creamos una tercera función que va a aplicar el oráculo (incluido el último paso) y luego aplicará su inverso

In [ ]:
def oracle_computation(qc,nod,lin,ver):
    oracle(qc,nod,lin,ver)
    qc.z(ver[0]) #Marcamos la respuesta
    inverse_oracle(qc,nod,lin,ver)

### El algoritmo de Grover completo

Finalmente, solo necesitamos nuestro operador de difusión para tener el algoritmo de Grover completo

In [ ]:
def diffuser(qc,nod,n):
#Primer paso
    for i in range(n):
        qc.h(nod[i])
        qc.x(nod[i])
#Segundo paso
    qc.h(nod[n-1])
    qc.mcx([nod[0],nod[1],nod[2]],nod[n-1])
    qc.h(nod[n-1])
#Tercer paso
    for i in range(n):
        qc.x(nod[i])
        qc.h(nod[i])

In [ ]:
nod = QuantumRegister(4, name="nod")
lin = QuantumRegister(4, name="lin")
ver = QuantumRegister(1, name="ver")
c = ClassicalRegister(4)

qc = QuantumCircuit(nod,lin,ver,c)

#Inicializamos el espacio de búsqueda
for i in range(4):
    qc.h(nod[i])

#Aplicamos el oráculo completo y el difusor 2 veces
for i in range(2):
    oracle_computation(qc,nod,lin,ver)
    diffuser(qc,nod,4)

#Medimos el registro de los nodos

qc.measure(nod,c)

#Ejecutamos nuestro circuito
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100)
counts = job.result().get_counts(qc)
print(counts)
#Graficamos nuestros resultados en un histograma
plot_histogram(counts)

Como puedes ver, las cadenas de bits que más veces medimos resultaron ser $0110$ y $1001$, las cuales corresponden a los nodos 0 y 3 de un color (rojo o azul) y los nodos 1 y 2 de otro color (azul o rojo). 

### Ejemplo #2

Ahora, veamos qué sucede cuando una gráfica no es bipartita. Tomemos la siguiente gráfica como un ejemplo:

<img src="Imagenes/triangulo.png" width="20%">

Como podemos ver, no hay forma de pintar este triángulo de manera que todas nuestras líneas conectoras se encuentren entre dos nodos de distinto color. Veamos qué sucede si aplicamos nuestro algoritmo de Grover a este problema. En este caso, y para poder usar un operador de difusión general, vamos a seleccionar:

1. Qubits 0-2 como los nodos.
1. Qubits 3-5 como las líneas conectoras.
1. Qubit 6 como el de verificación.

In [ ]:
def oracle2(qc):
#Primero, revisamos las líneas conectoras
    qc.cx(0,3) #Empezamos con la línea conectora de los nodos 0-1
    qc.cx(1,3)
    
    qc.cx(0,4) #Línea conectora de los nodos 0-2
    qc.cx(2,4)
    
    qc.cx(1,5) #Línea conectora de los nodos 1-2
    qc.cx(2,5)
    
#Ahora revisamos que todas las líneas conectoras se encuentren en el estado 1

    qc.mcx([3,4,5],6)

In [ ]:
def inverse_oracle2(qc):

    qc.mcx([3,4,5],6)
    
    qc.cx(0,3) #línea conectora de los nodos 0-1
    qc.cx(1,3)
    
    qc.cx(0,4) #Línea conectora de los nodos 0-2
    qc.cx(2,4)
    
    qc.cx(1,5) #Línea conectora de los nodos 1-2
    qc.cx(2,5)

In [ ]:
def oracle_computation2(qc):
    oracle2(qc)
    qc.z(6) #Marcamos la respuesta
    inverse_oracle2(qc)

In [ ]:
def diffuser(qc,n):
#Primer paso
    for i in range(n):
        qc.h(i)
        qc.x(i)
#Segundo paso
    qc.h(n-1)
    qc.mcx(list(range(n-1)),n-1)
    qc.h(n-1)
#Tercer paso
    for i in range(n):
        qc.x(i)
        qc.h(i)

In [ ]:
qc = QuantumCircuit(7,3)

#Inicializamos el espacio de búsqueda
for i in range(3):
    qc.h(i)

#Aplicamos el oráculo completo y el difusor 1 vez
oracle_computation2(qc)
diffuser(qc,3)

#Medimos los nodos

qc.measure([0,1,2],[0,1,2])

#Ejecutamos nuestro circuito
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100)
counts = job.result().get_counts(qc)
print(counts)
#Graficamos nuestros resultados en un histograma
plot_histogram(counts)